# AI Saturday: Prep Session 2

## Simple Backpropagation Demo


This is a notebook for trying backpropagation on your own. 
Modified from Stanford's CS231n CCN for visual recognition class

![backnorm](https://kratzert.github.io/images/bn_backpass/BNcircuit.png)

In [1]:
import sys
print(f'This is running: {sys.version}')
import numpy as np
import pandas as pd

# If you're going to be running this script in the term or as a script 
# set the matplotlib backend so figures can be saved in the background
# (uncomment the lines below if you are using a headless server)
# import matplotlib
# matplotlib.use("Agg")
# import matplotlib.pyplot as plt

import plotly
plotly.offline.init_notebook_mode(connected=True) 
import plotly.offline as py #similiar to ggplot's qplot
import plotly.graph_objs as go #like R's geoms

This is running: 3.6.2 |Anaconda, Inc.| (default, Sep 30 2017, 18:42:57) 
[GCC 7.2.0]


In [2]:
#Generate non-linear data

N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D)) # data matrix (each row = single example)
y = np.zeros(N*K, dtype='uint8') # class labels
for j in range(K):
  ix = range(N*j,N*(j+1))
  r = np.linspace(0.0,1,N) # radius
  t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
  X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
  y[ix] = j

A quick preview of the X and Y's

In [3]:
print(type(X))
print(X[:5,])
np.unique(y)

<class 'numpy.ndarray'>
[[-0.          0.        ]
 [-0.00204491  0.00989185]
 [ 0.00144815  0.02015005]
 [ 0.00500471  0.0298869 ]
 [ 0.02228076  0.0337054 ]]


array([0, 1, 2], dtype=uint8)

In [4]:
#Lets plot
trace = go.Scatter(
    x = X[:,0],
    y= X[:,1], 
    mode='markers', # similar to geom_point
    marker = dict(  # feels similar to aes
        color = y 
    )
)
data = [trace]
py.iplot({"data":data})

### Linear Classifier

$$f(x_i, W, b) =  W x_i + b$$

N examples, 300; 100 each, (each with a dimensionality D, in our case 2 dimension) and K distinct categories (3). Where ${W}$ is the weight matrix and ${b}$ the bias

In images it could be 3 dimensions for RGB

![classifier image](http://cs231n.github.io/assets/imagemap.jpg)

In [5]:
W = 0.01 * np.random.randn(D,K)
b = np.zeros((1,K))
print(f'Shape of the Weight Matrix W {W.shape}, fill with random numbers')
W
print(f'Shape of the bias vector {b.shape}, its currently all zeros')
b

Shape of the Weight Matrix W (2, 3), fill with random numbers
Shape of the bias vector (1, 3), its currently all zeros


array([[ 0.,  0.,  0.]])

In [6]:
scores = np.dot(X, W) + b

Still not in terms of probability so we convert to probability ${p}$

$$p_k = \frac{e^{f_k}}{ \sum_j e^{f_j} }$$

In [11]:
num_examples = X.shape[0]

# get unnormalized probabilities
exp_scores = np.exp(scores)

# normalize them for each example
probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

#Print the probabilities of the 1st 5 examples
probs[:5,:]

array([[ 0.33333333,  0.33333333,  0.33333333],
       [ 0.33332346,  0.33335769,  0.33331885],
       [ 0.33330726,  0.33338116,  0.33331158],
       [ 0.33329163,  0.33340335,  0.33330502],
       [ 0.33326866,  0.33340699,  0.33332435]])

$$L_i =-\log\left(p_{y_i}\right)  \hspace{0.5in} \text{or equivalently} \hspace{0.5in} L_i = -f_{y_i} + \log\sum_j e^{f_j}$$

In [8]:
correct_logprobs = -np.log(probs[range(num_examples),y])
correct_logprobs.shape, probs[range(num_examples),y][:5], correct_logprobs[:5]

((300,),
 array([ 0.33333333,  0.33332346,  0.33330726,  0.33329163,  0.33326866]),
 array([ 1.09861229,  1.09864192,  1.09869051,  1.0987374 ,  1.09880632]))

In [9]:
# compute the loss: average cross-entropy loss and regularization
reg = 0.2 # lambda or L2, the higher the value the more you penalise larger weights
data_loss = np.sum(correct_logprobs)/num_examples
data_loss
reg_loss = 0.5*reg*np.sum(W*W)
loss = data_loss + reg_loss
loss

1.0981871290713776

![forwardBackword](https://kratzert.github.io/images/bn_backpass/chainrule_example.PNG)

In [10]:
%load_ext version_information
%version_information pandas, numpy, plotly, matplotlib

Software versions
Python 3.6.2 64bit [GCC 7.2.0]
IPython 6.1.0
OS Linux 4.10.0 42 generic x86_64 with debian stretch sid
pandas 0.21.0
numpy 1.13.3
plotly 2.2.3
matplotlib 2.1.0
Fri Dec 29 15:07:31 2017 +08